## Übung zu Kapitel 11

In dieser Übung wenden wir die eingeführten statistischen Methoden zum besseren Verständnis an. Hierzu vergleichen wir verschiedene Klassifikationsmodelle auf den [Irisdaten](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). 

### Mehrfaches Training

Erstellen Sie fünf zufällige Aufteilungen der Irisdaten in je 50% Trainingsdaten und 50% Testdaten. Trainieren Sie ein 5-Nearest-Neighbor-Modell und einen Random Forest (mit 100 Random Trees) für jede dieser Aufteilungen. Berechnen Sie die Güte für jedes dieser Modelle mit MCC und erstellen Sie für jedes Modell eine Liste mit den Ergebnissen.

### Statistischer Vergleich

Vergleichen Sie das arithmetische Mittel, die Standardabweichung, den Median, das Minimum und das Maximum des MCC für beide Algorithmen. Benutzen Sie geeignete statistische Tests, um zu bestimmen, ob der Unterschied signifikant ist. Falls der Unterschied signifikant ist, berechnen Sie Cohens $d$, um die Effektstärke zu bestimmen. Berechnen Sie außerdem das 95%-Konfidenzintervall des arithmetischen Mittels von MCC.

### Mehr Vergleiche

Wiederholen Sie diese Auswertung mit 10, 50 und 100 zufälligen Aufteilungen der Irisdaten. Wie verändern sich die Ergebnisse?

Der folgende Quelltext erledigt alle Aufgaben gleichzeitig. 

In [1]:
import numpy as np

from sklearn.datasets import load_iris
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy import stats
from statistics import mean, stdev
from math import sqrt
from scipy.stats import norm
from textwrap import TextWrapper

wrapper = TextWrapper(width=65)


def wrap_print(string):
    print('\n'.join(wrapper.wrap(string)))


X, Y = load_iris(return_X_y=True)

knn = KNeighborsClassifier(5)
rf = RandomForestClassifier(n_estimators=100)

scores_rf = []
scores_knn = []
for repetitions in [5, 10, 50, 100]:
    wrap_print("----------------------------------------------------------------")
    wrap_print(
        "Schätze Güte von KNN und Random Forest mit %i Train/Test Splits" % repetitions)
    for i in range(0, repetitions):
        # bootstrap loop
        X_train, X_test, Y_train, Y_test = train_test_split(
            X, Y, test_size=0.50, stratify=Y)
        rf.fit(X_train, Y_train)
        Y_pred = rf.predict(X_test)
        scores_rf.append(matthews_corrcoef(Y_test, Y_pred))
        knn.fit(X_train, Y_train)
        Y_pred = knn.predict(X_test)
        scores_knn.append(matthews_corrcoef(Y_test, Y_pred))

    s1 = scores_rf
    s2 = scores_knn

    def calc_ci(mu, sd, n, cl):
        dev = norm.ppf((1-cl)/2)*sd/sqrt(n)
        return mu+dev, mu-dev

    s1_lower, s1_upper = calc_ci(np.mean(s1), np.std(s1), len(s1), 0.95)
    s2_lower, s2_upper = calc_ci(np.mean(s2), np.std(s2), len(s2), 0.95)

    print()
    print("Random Forest: mean   = %.4f [%.4f,%.4f]" % (
        np.mean(s1), s1_lower, s1_upper))
    print("               sd     = %.4f" % np.std(s1))
    print("               median = %.4f" % np.median(s1))
    print("               min    = %.4f" % np.min(s1))
    print("               max    = %.4f" % np.max(s1))
    print("KNN:           mean   = %.4f [%.4f,%.4f]" % (
        np.mean(s2), s2_lower, s2_upper))
    print("               sd     = %.4f" % np.std(s2))
    print("               median = %.4f" % np.median(s2))
    print("               min    = %.4f" % np.min(s2))
    print("               max    = %.4f" % np.max(s2))
    print()

    alpha = 0.005
    pval_shapiro1 = stats.shapiro(s1)[1]
    pval_shapiro2 = stats.shapiro(s2)[1]

    print('p-value des Shapiro-Wilk test für den Random Forest:   %.4f' %
          pval_shapiro1)
    if pval_shapiro1 > alpha:
        wrap_print('Der Test ergibt das die Daten wahrscheinlich normalverteilt sind und wir können die Nullhypothese nicht ablehnen mit einem Signifikanzniveau von alpha=%.3f' % alpha)
    else:
        wrap_print('Der Test ergibt das die Daten wahrscheinlich nicht normalverteilt sind und wir lehnen die Nullhypothese ab mit einem Signifikanzniveau von alpha=%.3f' % alpha)

    print()
    print('p-value des Shapiro-Wilk test für KNN: %.4f' % pval_shapiro2)
    if pval_shapiro2 > alpha:
        wrap_print('Der Test ergibt das die Daten wahrscheinlich normalverteilt sind und wir können die Nullhypothese nicht ablehnen mit einem Signifikanzniveau von alpha=%.3f' % alpha)
    else:
        wrap_print('Der Test ergibt das die Daten wahrscheinlich nicht normalverteilt sind und wir lehnen die Nullhypothese ab mit einem Signifikanzniveau von alpha=%.3f' % alpha)
        
    print()

    if pval_shapiro1 > alpha and pval_shapiro2 > alpha:
        wrap_print("Beide Lösungen sind wahrscheinlich normalverteilt. Benutze Welch's t-test.")
        pval_equal = stats.ttest_ind(s1, s2, equal_var=False)[1]
        print()
        print("p-value von Welch's t-test: %f" % pval_equal)
    else:
        wrap_print(
            "Mindestens eine Verteilung wahrscheinlich nicht normalverteilt. Benutze den Mann-Whitney-U Test.")
        pval_equal = stats.mannwhitneyu(s1, s2, alternative='two-sided')[1]
        print()
        print("p-value vom Mann-Whitney-U Test: %f" % pval_equal)

    if pval_equal > alpha:
        wrap_print('The Test ergibt, dass die Mittelwerte vermutlich gleich sind und wir können die Nullhypothese nicht ablehnen mit einem Signifikanzniveau von alpha=%.3f' % alpha)
    else:
        wrap_print('The Test ergibt, dass die Mittelwerte vermutlich nicht gleich sind und wir lehnen die Nullhypothese ab mit einem Signifikanzniveau von alpha=%.3f' % alpha)
        s = sqrt(((len(s1)-1)*stdev(s1)**2 + (len(s2)-1)
                  * stdev(s2)**2)/(len(s1)+len(s2)-2))
        cohens_d = (mean(s1) - mean(s2)) / s

        if abs(cohens_d) < 0.01:
            effsizestr = "Vernachlässigbarer"
        elif abs(cohens_d) < 0.2:
            effsizestr = "Sehr kleiner"
        elif abs(cohens_d) < 0.5:
            effsizestr = "Kleiner"
        elif abs(cohens_d) < 0.8:
            effsizestr = "Mittlerer"
        elif abs(cohens_d) < 1.2:
            effsizestr = "Großer"
        elif abs(cohens_d) < 2:
            effsizestr = "Sehr großer"
        else:
            effsizestr = "Riesiger"

        wrap_print("Effektstärke (Cohen's d): %.3f - %s Effekt" %
                   (cohens_d, effsizestr))

----------------------------------------------------------------
Schätze Güte von KNN und Random Forest mit 5 Train/Test Splits

Random Forest: mean   = 0.9249 [0.9052,0.9446]
               sd     = 0.0225
               median = 0.9200
               min    = 0.9022
               max    = 0.9600
KNN:           mean   = 0.9562 [0.9332,0.9792]
               sd     = 0.0262
               median = 0.9600
               min    = 0.9210
               max    = 1.0000

p-value des Shapiro-Wilk test für den Random Forest:   0.4010
Der Test ergibt das die Daten wahrscheinlich normalverteilt sind
und wir können die Nullhypothese nicht ablehnen mit einem
Signifikanzniveau von alpha=0.005

p-value des Shapiro-Wilk test für KNN: 0.7614
Der Test ergibt das die Daten wahrscheinlich normalverteilt sind
und wir können die Nullhypothese nicht ablehnen mit einem
Signifikanzniveau von alpha=0.005

Beide Lösungen sind wahrscheinlich normalverteilt. Benutze
Welch's t-test.

p-value von Welch's t-test: 

Wie man sieht sind die Werte der Algorithmen relativ Stabil, die Ergebnisse der statistischen Auswertung hängen jedoch von der Anzahl der Iterationen ab. Mit fünf Iterationen sehen wir noch keinen signifikanten Unterschied. Dies ändert sich jedoch mit größeren Stichproben durch mehr Iterationen. Auch das Konfidenzinterval ist zu Beginn noch relativ groß und schrumpft durch mehr Iterationen, da die Sicherheit der Schätzung höher wird. 

Dies demonstriert einen wichtigen Aspekt von statistischer Analyse: Man kann zu kleinen Stichproben nicht trauen. 

Man sieht ebenfalls, dass die Effektstärke irreführend sein kann. Die Analyse zeigt eine mittlere Effektstärke. Der absolute Unterschied zwischen den Modellen ist mit 2% jedoch relativ klein. Die Effektstärke ist dennoch sehr hoch, da die Standardabweichung der Ergebnisse sehr klein ist. Grundsätzlich sollte man mit Effektstärken im Fall von sehr kleinen Standardabweichungen vorsichtig umgehen und diese nicht überinterpretieren. 